## Satellite Data

This script outputs the satellite-based rasters into the Google Earth Engine Cloud.

Inputs:

    - From MAPBIOMAS:
        - secondary forest age
        - land use land cover
        - fire
    - From ESA CCI Biomass:
        - biomass (Mg C/hectare)

Outputs:

    - From MAPBIOMAS:
        - last observed land use type
        - number of years under each land use type
        - number of fires
        - time since last fire
        - fallow period length


In [1]:
import ee
import geemap

# Authenticate to Earth Engine
try:
  ee.Initialize(project='ee-ana-zonia')
except Exception as e:
  ee.Authenticate()
  ee.Initialize(project='ee-ana-zonia')

first_year = 1985
last_year = 2020

from utils import export_image
from utils import map_image

In [2]:
# import ages from MapBiomas
age = ee.Image('projects/mapbiomas-workspace/public/collection8/mapbiomas_collection80_secondary_vegetation_age_v1').select('secondary_vegetation_age_2020')

# region of interest is the entire country
roi = ee.FeatureCollection("projects/ee-ana-zonia/assets/biomes_ibge").geometry().dissolve()
# For specific biomes, the feature "id" is:
    # 18413 = Amazon
    # 18414 = Caatinga
    # 18415 = Cerrado
    # 18416 = Mata Atlantica
    # 18417 = Pampa
    # 18418 = Pantanal

# Load images from MapBiomas Collection 8 for Land Use Land Cover and Burned Area
lulc = ee.Image("projects/mapbiomas-workspace/public/collection8/mapbiomas_collection80_integration_v1") \
  .select([f"classification_{year}" for year in range(first_year, last_year+1)]).byte()
lulc = lulc.updateMask(age)

fire = ee.Image("projects/mapbiomas-workspace/public/collection7_1/mapbiomas-fire-collection2-annual-burned-coverage-1") \
  .select([f"burned_coverage_{year}" for year in range(first_year, last_year)]).byte()
fire = fire.updateMask(age)

## Removing unwanted pixels

### Undesired histories

Some land use categories are not relevant to the model (such as rocky surfaces or mangroves)

All pixels with **at least one observation of the undesired land use history** are used to make a mask, to leave behind only pixels with occurrences of only desired land use types.


Land use types we are interested in:

    3 = forest
    15 = pasture
    39 = soy
    20 = sugar cane
    40 = rice
    62 = cotton
    41 = other temporary crop
    46 = coffee
    47 = citrus
    35 = palm oil
    48 = other perennial crop
    9 = forest plantationantation

In [4]:
# List the categories that are DESIRED to be maintained
desired_values = ee.List([3, 15, 39, 20, 40, 62, 41, 46, 47, 35, 48, 9, 21])
mask_all_ones = ee.List.repeat(1, desired_values.size())

# For each band, convert pixels with desired land use types to 1 - undesired types to zero
def remap_band(band_name):
    band = lulc.select(ee.String(band_name))
    new_band = band.remap(desired_values, mask_all_ones, 0)
    return new_band.rename(ee.String(band_name))

# Map the function over the band names
remapped_image = lulc.bandNames().map(remap_band)
# make mask by adding all pixels - the ones that add up to 36 have all pixels with desired categories
remapped_image = ee.ImageCollection(remapped_image).toBands()
desired_mask = remapped_image.reduce('sum').eq(lulc.bandNames().size().getInfo())

lulc = lulc.updateMask(desired_mask)
age = age.updateMask(desired_mask)


### Isolated pixels

In the map, there were isolated pixels, often around the edges of forest patches. These would likely be due to misclassification, or follow different behaviors due to edge effects.

To avoid this issue, a kernel is applied here to include only secondary forest patches that are one hectare or larger.

In [5]:
kernel = ee.Kernel.square(radius = 100, units = 'meters')
# convert non-forest pixels from NA to zero
age_zeroes = age.unmask(0)
# check what is the most frequent value within each hectare - if it's zero, it means the pixel is surrounded by non-forest cover
age_mask = age_zeroes.reduceNeighborhood(reducer = ee.Reducer.mode(), kernel = kernel)

# mask age raster
age = age.updateMask(age_mask)

### Pixels that were forested in 1985 (35 years of age)

- Make a mask with regrowing patches that were forested in 1985 (keep only the patches that were forested in 1985, then subsequently deforested, and regrew after that)
- Remove from "age" the forests that are older than 33 (they have no history as we have no information before 1985)


In [6]:
age = age.updateMask(age.lt(34)).updateMask(lulc.select('classification_1985').eq(3))

## Land Use Land Cover
### Total sum of years under each desired land use type

For each desired land use type, the total occurrences per pixel are summed

In [7]:
# keeping only the land use history of the pixels with ages that we are studying
lulc = lulc.updateMask(age)

# Listing land use types that are NOT forest
LU_index = [15, 39, 20, 40, 62, 41, 46, 47, 35, 48, 9, 21]

LU_sum = ee.Image()

for val in LU_index:
  lulc_val = lulc.eq(val)
  num_cells = lulc_val.reduce(ee.Reducer.sum()).rename(f'lulc_sum_{val}')
  LU_sum = LU_sum.addBands(num_cells)

# removes the first (empty) band
LU_sum = LU_sum.slice(1).byte()

### Last Land Use Type

Finds what was the last land use type observed before abandonment.

In [8]:
years = range((first_year + 1), last_year)

last_LU = ee.Image()

# for each forest age, mask the lulc of the year immediately preceding abandonment
for yr in years:
    year = f'classification_{yr}'
    lu_yr = lulc.select(year)
    age_mask = age.eq(last_year - yr)
    last_LU = last_LU.addBands(lu_yr.updateMask(age_mask))#.updateMask(lu_yr.neq(3))

# remove first empty band
last_LU = last_LU.slice(1)

# merge all images by adding them up
last_LU = last_LU.reduce(ee.Reducer.sum()).rename('last_LU')


### Fallow period length

Finds the total number of years that land was left fallow (classified as forest before regrowth)

In [10]:
nat_cover = lulc.updateMask(lulc.eq(3))
total_nat_cover = nat_cover.reduce(ee.Reducer.sum())
fallow = total_nat_cover.subtract(age).rename('fallow')

## Fire

### Total number of fires per pixel

Each burned pixel contains the value of the land use type that burned (similar to desired_values specified in the land use se- unburned pixels are zero.

Note that fire has different transform than lulc, and the projections will be adjusted when exporting.

In [283]:
# Making a fire mask with pixels of value 1 for burned pixels and 0 otherwise instead:
fire = fire.gt(0)
num_fires = fire.reduce(ee.Reducer.sum()).rename('num_fires').byte()

In [291]:
# how many years ago was EACH fire? #############################
# Get the number of bands
num_bands = fire.bandNames().size()
# Create a sequence of numbers from 1 to num_bands
years_ago = ee.List.sequence(1, num_bands)
years_ago = years_ago.reverse()

# Map over the image and set values based on the band index
constant_images = ee.ImageCollection.fromImages(
    years_ago.map(lambda year: ee.Image.constant(year))).toBands()

time_since_all_fires = fire.multiply(constant_images)

old_names = time_since_all_fires.bandNames().getInfo()
new_names = [name.replace('burned_coverage', 'time_since_fire') for name in old_names]
time_since_all_fires = time_since_all_fires.select(old_names).rename(new_names)

In [300]:
# how many years ago was the LAST fire? #############################
last_fire = time_since_all_fires.reduce(ee.Reducer.lastNonNull()).rename('last_fire').byte()

In [295]:
# get fire frequency data from Mapbiomas - double check it.
fire_freq = ee.Image("projects/mapbiomas-workspace/public/collection7_1/mapbiomas-fire-collection2-fire-frequency-1").clip(roi)
fire_freq = fire_freq.select('fire_frequency_1985_2020').clip(ecoregions.first())

## Biomass

In [5]:
# biomass for 2020 comes from CCI Biomass
biomass = ee.Image("projects/ee-ana-zonia/assets/raw/biomass")
proj = biomass.projection().getInfo()
crs = proj['crs']
crsTransform = proj['transform']

In [6]:
# Reproject to 10m
biomass = biomass.reproject(crs=age.projection(), scale=10)
# Reaggregate to 30m (mean value)
biomass = biomass.reduceResolution(reducer=ee.Reducer.mean()).reproject(crs=age.projection())
# Mask only to regions with age greater than zero (secondary forests)
biomass = biomass.updateMask(age.mask()).int16().rename('agbd')

### Mature forest biomass

In [12]:
# Mask the image to keep only pixels with natural vegetation cover in all bands
# 3 = forest
# 6 = flooded forest
biomass = ee.Image("projects/ee-ana-zonia/assets/biomass")
ecoregions = (ee.FeatureCollection("RESOLVE/ECOREGIONS/2017").filterBounds(roi)
                .map(lambda feature: feature.intersection(roi)))

mature_cover = lulc.eq(3).Or(lulc.eq(6))
mature_mask = mature_cover.reduce(ee.Reducer.allNonZero())
mature_biomass = biomass.updateMask(mature_mask.mask())


# vis = {
#     'min': 0,
#     'max': 400,
#     'palette': ['blue', 'red'],
# }

# Map = geemap.Map()
# Map.addLayer(mature_mask, {}, 'mature_mask')
# Map.addLayer(lulc_masked, {}, 'lulc_masked')
# Map.addLayer(mature_biomass, vis, 'mature_biomass')
# Map

In [13]:
# Compute the mean biomass values for mature forest per ecoregion.
median_mature = mature_biomass.reduceRegions(ecoregions, reducer = ee.Reducer.median(),\
                                             scale = 10000, crs = crs)

# Convert the FeatureCollection to an image.
median_mature = median_mature.reduceToImage(['median'], ee.Reducer.first())



## Export images as asset to Google Earth Engine

In [13]:
export_image(age, "age")
# export_image(last_LU, "last_LU")
# export_image(LU_sum, "LU_sum")
# export_image(last_fire, "last_fire")
# export_image(num_fires, "num_fires")
# export_image(age, "age")
# export_image(fallow, "fallow")
# export_image(lulc_masked, "lulc")
# export_image(biomass, "biomass_masked")
# export_image(median_mature, 'median_mature')

# export_image(mature_biomass, "mature_biomass")


In [17]:
LU_sum = ee.Image('projects/ee-ana-zonia/assets/LU_sum')
fallow = ee.Image('projects/ee-ana-zonia/assets/fallow')
last_fire = ee.Image('projects/ee-ana-zonia/assets/last_fire')
num_fires = ee.Image('projects/ee-ana-zonia/assets/num_fires')
frag = ee.Image('projects/ee-ana-zonia/assets/frag_2020')
med_mat = ee.Image('projects/ee-ana-zonia/assets/median_mature')
last_LU = ee.Image('projects/ee-ana-zonia/assets/last_LU')

land_use = LU_sum.addBands([last_LU, fallow, last_fire, num_fires, frag, med_mat])

ecoreg = ee.Image("projects/ee-ana-zonia/assets/raw/ecoregions_resolve")
indig = ee.Image("projects/ee-ana-zonia/assets/indig")
soil = ee.Image("projects/ee-ana-zonia/assets/soil")
protec = ee.Image("projects/ee-ana-zonia/assets/protec")
biomes = ee.Image("projects/ee-ana-zonia/assets/biome_br")
categorical = ecoreg.addBands([indig, soil, protec, biomes])

yearly_si = ee.Image("projects/ee-ana-zonia/assets/yearly_si").float()
mean_prec = ee.Image("projects/ee-ana-zonia/assets/mean_prec").float()
climate = yearly_si.addBands([mean_prec])

biomass = ee.Image('projects/ee-ana-zonia/assets/biomass_masked')
total_export = categorical.addBands([land_use, climate, biomass, age]).updateMask(age)

nat_cover = lulc.select('classification_1985')
nat_cover = nat_cover.updateMask((lulc.neq(1).neq(4)).neq(5)).lt(7)
total_export = total_export.updateMask(nat_cover)

In [18]:
img = total_export
name = "total_export_mat_1985"

# Create the export task
task = ee.batch.Export.image.toAsset(
    image = img,
    description = f'{name}',
    assetId = f'projects/ee-ana-zonia/assets/{name}',
    crs = 'EPSG:4326',
    scale = 30,
    maxPixels = 4e12,
    region=img.geometry()
)

# Start the export task
task.start()